In [11]:
!pip install onnx onnxruntime

In [12]:
import math
import onnx
import onnxruntime as ort
import numpy as np
from onnx import TensorProto

# Define input and output tensor names
input1_name = "X"
maxpool_output_name1 = "Y"
maxpool_output_name2 = "Indices"


# Create the ONNX model with maxpool operator
def create_maxpool_model(input_shape, output_shape, dtype, paddings):

    #Create "input-rank" input tensor
    input1 = onnx.helper.make_tensor_value_info(input1_name, dtype, input_shape)

    # Create output tensor (final result after maxpool operation)
    maxpool_output1 = onnx.helper.make_tensor_value_info(maxpool_output_name1, dtype, output_shape)
    maxpool_output2 = onnx.helper.make_tensor_value_info(maxpool_output_name2, TensorProto.INT64, output_shape)

    # Define clip node
    maxpool_node = onnx.helper.make_node(
        "MaxPool",
        inputs=[input1_name],
        outputs=[maxpool_output_name1, maxpool_output_name2],
        ceil_mode = 0,
        dilations=[1,1],
        kernel_shape=[3,3],
        pads=paddings,
        strides=[1, 1],
        auto_pad='NOTSET',
    )

    # Create the ONNX graph
    graph_def = onnx.helper.make_graph(
        [maxpool_node],
        "Test_MaxPool",
        [input1],
        [maxpool_output1, maxpool_output2],
    )

    # Create the ONNX model
    model = onnx.helper.make_model(graph_def, opset_imports=[onnx.helper.make_opsetid("", 22)]) # Explicitly set opset to 13
    model.ir_version = 10 
    onnx.checker.check_model(model)

    # Save the model
    onnx.save(model, "maxpool.onnx")

    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("maxpool.onnx")
    return session

def do_maxpool(x, session):
    # Run inference
    output = session.run(None, {input1_name: x})

    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '\n'))
    indices_f = (np.array2string(output[1], separator=',', max_line_width=np.inf).replace('\n', '\n'))

    # Display results
    print(f"X = \n{x_f}")
    print(f"Y = \n{y_f}")
    print(f"Indices = \n{indices_f}")

def compute_MaxPool_output_shape(input_shape, kernel_shape, pads, dilation, strides):

    dx0, dx1, dx2, dx3 = input_shape
    dw0, dw1 = kernel_shape


    myalpha = dx2 + pads[0] + pads[2]
    mybeta = dx3 + pads[1] + pads[3]
    mytheta = (dilation[0] * (dw0 - 1)) + 1
    mygamma = (dilation[1] * (dw1 - 1)) + 1
#   dY_2 = \left\lfloor{(dX_2 + pad\_shape[0] - dilations[0] * (kernel\_shape[0] - 1) - 1) / (strides[0] + 1)}\right\rfloor
    mydy2 = math.floor(((myalpha - (mytheta)) / strides[0]) + 1)
#   dY_3 = \left\lfloor{(dX_3 + pad\_shape[1] - dilations[1] * (kernel\_shape[1] - 1) - 1) / (strides[1] + 1)} \right\rfloor
    mydy3 = math.floor(((mybeta - (mygamma)) / strides[1]) + 1)
    
    output_shape = [dx0, dx1, mydy2, mydy3]

    return output_shape


def pad_4d_spatial_js(tensor, paddings):
    """
    Pads the last two dimensions (H, W) of a 4D tensor.
    paddings = (top, left, bottom, right)
    """
    N, C, H, W = tensor.shape
    top, left, bottom, right = paddings
    
    # 1. Calculate the new spatial dimensions
    new_H = H + top + bottom
    new_W = W + left + right
    
    # 2. Create the new 4D "canvas" of zeros
    # The Batch (N) and Channel (C) dimensions remain the same
#    padded_tensor = np.zeros((N, C, new_H, new_W), dtype=tensor.dtype)
    padded_tensor = np.full((N, C, new_H, new_W), -np.inf)
    
    # 3. Insert the original tensor into the padded one
    # We use ":" for N and C to select all batches and channels
    # We use slicing for the H and W dimensions
    padded_tensor[:, :, top : top + H, left : left + W] = tensor
    
    return padded_tensor

def extract_max_dilated_from_dilated_window(X_p, b, c, m, n, strides, dilations, dW):
    """
    Extracts the maximum value from a dilated window.
    
    Args:
        X_p: The 2D padded input array.
        m, n: Integer constants representing the current output position.
        strides: Tuple (s_h, s_w) for step size.
        dilations: Tuple (d_h, d_w) for spacing between window elements.
        dW: Tuple (dW0, dW1) for the window height and width.
    """
    dW0, dW1 = dW
    s_h, s_w = strides
    d_h, d_w = dilations
   
    # Calculate the base (top-left) coordinates based on the stride
    base_h = m * s_h
    base_w = n * s_w
    
    # Initialize max_val with the first element of the window (h=0, w=0)
    max_val = X_p[b, c, base_h, base_w]
    
    # Iterate through the window defined by dW0 and dW1
    for h in range(dW0):
        for w in range(dW1):
            # Calculate the current dilated coordinates
            curr_h = base_h + h * d_h
            curr_w = base_w + w * d_w
            
            # Extract the value at these coordinates
            val = X_p[b, c, curr_h, curr_w]
            
            # Update the maximum found so far
            if val > max_val:
                max_val = val
                
    return max_val

def MaxPool(X, kernel_shape, strides, dilation, pads):

    Y_dims = compute_MaxPool_output_shape(X.shape, kernel_shape, pads, dilation, strides)

    Y = np.zeros(Y_dims)

    X_p = pad_4d_spatial_js(X, pads)
    dX_p0, dX_p1, dX_p2, dX_p3 = X_p.shape
    dY_p0, dY_p1, dY_p2, dY_p3 = Y.shape

    xp_f = (np.array2string(X_p, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    print(f"MaxPool: X_p = \n{xp_f}")

    for b in range(dX_p0):
       for c in range(dX_p1): 
         for m in range(dY_p2):
           for n in range(dY_p3):
             Y[b, c, m, n] = extract_max_dilated_from_dilated_window(X_p, b, c, m, n, strides, dilation, kernel_shape)

    Y_f = (np.array2string(Y, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    print(f"MaxPool: Y = \n{Y_f}")


np.set_printoptions(precision=None, floatmode='fixed')

## Nominal Cases

In [18]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
pads = [0,0,0,0]
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type, pads)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], pads)

X = 
[[[[ 1.79600868, 3.17047056, 3.24585250,-0.70210242, 0.21493457, 3.88964306, 2.83984821, 7.66431059],
   [-0.64609348, 5.68134645, 0.25121777,-0.57023099, 0.19189985, 6.35651387, 2.40120956, 3.44485406],
   [ 3.90387908, 2.42988301, 3.58786652, 2.39758521, 2.99846054,10.92699375, 4.50539251, 0.59171866],
   [ 2.87879519, 3.01687926, 2.37521792, 3.98320962, 6.39730190, 4.47257121, 0.92853818, 2.01846954],
   [ 4.59294020, 0.83082117, 0.12288230, 4.91708411, 4.55202322, 7.22642684, 1.96294360, 4.72855007],
   [ 5.98531149,-0.03705777, 0.62739196, 3.16546247, 5.73444619, 3.03724337, 2.03938838, 2.04420144],
   [-0.45958542, 5.71864670, 3.28681662, 0.87607173, 4.43693032, 1.38606641, 3.77389501, 7.60625193],
   [ 1.52890288, 7.84946994, 6.03473822, 2.41944794, 0.72211592, 2.87885244, 1.42214040, 1.74029770]]]]
Y = 
[[[[ 5.68134645, 5.68134645, 3.58786652,10.92699375,10.92699375,10.92699375],
   [ 5.68134645, 5.68134645, 6.39730190,10.92699375,10.92699375,10.92699375],
   [ 4.59294020,

In [19]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
pads = [1,0,1,0]
session = create_maxpool_model(x.shape, [1, 1, 8, 6], onnx_type, pads)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], pads)

X = 
[[[[ 5.13036980,-0.75398416, 4.56682130, 3.98975816, 2.34237621, 1.07973749, 4.45731956,-0.81557314],
   [ 0.27041389, 0.86839577, 1.41787814, 2.40521624, 0.61740179, 1.13739145, 2.33768741, 6.12945841],
   [ 2.24232462, 3.60132765, 5.11577101, 2.36663167,-0.78807371, 3.85064745, 0.33751639, 4.79362935],
   [-2.93080046, 6.06238572, 2.95884444, 4.09228087, 2.97917384, 4.44342584, 2.97723800, 6.89381799],
   [ 4.06489224, 3.71419092, 1.19594802, 2.25922909,-2.62004130,-0.62712872,-1.09160243, 5.54184707],
   [ 2.38458137, 3.08885415, 4.13964478, 5.76753808, 3.86394633, 3.24441708, 1.53291187,-2.09198368],
   [ 5.96249215, 4.45656932, 0.94222963, 4.99413990,-1.28668362, 2.56921295, 3.19327014, 5.54119022],
   [ 2.64363945, 3.71308955, 4.37333373, 1.49939608, 3.42357271, 0.26289900, 3.94289309, 6.77464645]]]]
Y = 
[[[[5.13036980,4.56682130,4.56682130,3.98975816,4.45731956,6.12945841],
   [5.13036980,5.11577101,5.11577101,3.98975816,4.45731956,6.12945841],
   [6.06238572,6.06238572,5.

In [20]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
pads = [0,1,0,1]
session = create_maxpool_model(x.shape, [1, 1, 6, 8], onnx_type, pads)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], pads)

X = 
[[[[-1.57535081, 0.19815172, 3.94942444, 4.05565170, 5.22354828, 3.75352392, 5.09895869, 1.82546961],
   [ 0.96238148,-0.94115081, 4.12284764, 5.19764488,-0.22567474, 1.67775259, 2.25931363, 4.54326553],
   [ 5.06451827,-1.35829665, 1.97454008, 2.93941081, 2.02587214, 4.88640497, 5.11783828, 1.71474916],
   [ 2.67883287, 2.36978894, 1.82021970, 6.48741245, 3.19498230,-2.92284077, 7.60259205, 7.65933107],
   [ 0.52292969, 5.86192032, 4.26640835, 5.78413283, 2.76844640, 1.21281288, 1.49251739,-0.04344814],
   [ 5.43127052, 1.32332627, 5.89413405, 4.54178421,-2.02457545, 4.09267119,-1.90543088, 5.44447556],
   [ 3.68443649, 3.41901403, 3.61283468, 3.76654174, 5.21036510, 1.48148403, 3.19905972, 3.34385306],
   [ 1.29208975, 2.03278919,-0.42120315, 4.98724429, 1.76065030, 2.17176464, 3.04800854, 0.40389047]]]]
Y = 
[[[[5.06451827,5.06451827,5.19764488,5.22354828,5.22354828,5.22354828,5.11783828,5.11783828],
   [5.06451827,5.06451827,6.48741245,6.48741245,6.48741245,7.60259205,7.659331

In [21]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
pads = [1,1,1,1]
session = create_maxpool_model(x.shape, [1, 1, 8, 8], onnx_type, pads)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], pads)

X = 
[[[[ 4.98607334, 4.43451312, 5.46418744, 7.96062541, 7.32356011, 0.83885246, 2.84992185, 3.18479986],
   [ 1.60812762, 1.65416875,-1.66700338, 2.83156972, 3.78214066, 2.51950782, 3.32110566, 1.74367219],
   [ 3.60167983, 1.44913491, 1.55946730,-0.09510786, 1.06540049, 4.26294296,-3.73937871,-0.14324033],
   [ 4.57257058, 2.88705675, 7.42822107, 2.87263169, 1.44952849,-0.21843020, 3.39916466,-0.96708609],
   [ 2.73243618, 2.42057721, 1.09960891, 2.18663321, 2.80399063, 2.29423523, 1.84706645, 3.45780033],
   [ 2.26660083, 4.35503051, 2.95268915, 4.23509706, 0.29842976, 5.18785400, 3.64514241, 3.84870089],
   [ 9.92767698, 2.58206290, 4.41543398, 3.32662113, 6.45969103,-0.02747886, 3.39070703, 2.31965640],
   [ 4.71011400, 5.05292920,-0.72234879, 0.30617864, 6.38084397, 2.64023278, 2.03306703, 3.86347293]]]]
Y = 
[[[[4.98607334,5.46418744,7.96062541,7.96062541,7.96062541,7.32356011,3.32110566,3.32110566],
   [4.98607334,5.46418744,7.96062541,7.96062541,7.96062541,7.32356011,4.262942

In [22]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
pads = [1,0,0,0]
session = create_maxpool_model(x.shape, [1, 1, 7, 6], onnx_type, pads)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], pads)

X = 
[[[[ 3.75975839, 0.63218521, 8.35759425, 4.61511272, 8.51478077, 6.81711875, 4.63548504, 3.52561379],
   [-1.32765504,-0.15931693, 2.00709488, 1.34298770, 1.46112165, 1.55408602, 4.90735235, 5.58452782],
   [ 4.93443325, 2.83761657, 5.75154303, 3.34446519, 5.02084812, 4.84546163, 1.28441732, 3.22493839],
   [ 0.77167803,-0.27540048, 1.37917745, 1.38754982, 1.19622917, 5.35550624, 6.66358844, 4.83410033],
   [ 0.99303323,-0.29535120, 1.95815551, 4.43278847, 0.19959602, 0.57786140, 2.15687216, 6.19514133],
   [-0.29973321, 1.87410672, 3.68007141, 4.52717912, 1.84257320, 2.83896877, 3.30248328, 3.85395828],
   [ 3.74535247, 5.72413985, 7.60905102, 0.26781042, 0.85889725, 2.21418779, 4.92636673,-0.56387039],
   [-0.03253864, 4.93199984, 4.68944788, 4.35129000, 4.00183387, 5.58044461, 2.40325486, 2.06543036]]]]
Y = 
[[[[8.35759425,8.35759425,8.51478077,8.51478077,8.51478077,6.81711875],
   [8.35759425,8.35759425,8.51478077,8.51478077,8.51478077,6.81711875],
   [5.75154303,5.75154303,5.

In [23]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
pads = [0,0,1,0]
session = create_maxpool_model(x.shape, [1, 1, 7, 6], onnx_type, pads)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], pads)

X = 
[[[[ 6.68758997, 3.44034635, 3.06023204,-2.17719825, 1.32991274, 6.48189503, 0.62419768, 2.76308118],
   [-2.20121786, 4.86027483, 0.58905797, 6.38904686, 2.98429462, 0.77338476,-1.60887085, 1.63812894],
   [ 6.81845954, 6.22773524, 0.58155682, 5.36573872,-1.18755163, 6.70753747,-0.50645250, 1.95861228],
   [ 4.97209458, 1.46070623, 3.81461452, 0.96006355, 0.89146504, 3.29981580, 0.51805791, 5.47982492],
   [10.28648831, 3.97566445, 2.29908000, 1.19967425, 5.38379523, 2.84386909, 4.06605147, 1.29910292],
   [ 5.14187115, 1.58952241, 5.01212996, 6.00465687, 7.99565434,-2.08399700, 5.81166604, 0.76170753],
   [ 3.07093120, 6.47080374,-0.52684661, 4.40289975,-4.30620774, 3.63044199, 2.59682099, 2.11699694],
   [ 2.81897369, 2.81738171, 4.60509516, 4.90102861, 4.98890190, 3.87693076, 3.37442063, 2.14076499]]]]
Y = 
[[[[ 6.81845954, 6.38904686, 6.38904686, 6.70753747, 6.70753747, 6.70753747],
   [ 6.81845954, 6.38904686, 6.38904686, 6.70753747, 6.70753747, 6.70753747],
   [10.28648831,

In [24]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
pads = [0,1,0,0]
session = create_maxpool_model(x.shape, [1, 1, 6, 7], onnx_type, pads)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], pads)

X = 
[[[[ 3.81298232, 0.34283475, 1.14314620, 2.97358787, 5.66563291, 4.86075205, 5.58519916, 3.69097858],
   [ 1.32598305, 6.83778017,11.80664148, 7.32146590, 2.68302092, 3.82571716, 6.76203895, 5.49982725],
   [ 6.01340078,-0.15222185, 7.03114049, 1.63563753, 1.34102856, 1.92684096, 3.91260788, 4.19297329],
   [ 0.18409862, 6.85141257, 0.31444599, 2.90975693, 3.34028908, 2.56295336, 6.16126852, 3.42714257],
   [ 4.54961394, 5.90543393, 0.86876104, 2.08616746, 6.39554545, 3.66527199, 2.51528368, 3.79772345],
   [-0.55021176, 2.32133683, 0.74765162, 3.38853220, 6.89484055, 3.78284987, 2.88039705, 5.93622064],
   [ 6.88611291, 0.73910469, 0.67937796,-0.38731361, 3.33262262, 2.54048657, 5.80065684,-1.95613297],
   [ 6.41852181, 3.43122619, 5.76366891, 0.77693951, 4.19584751, 4.94815978, 3.27727868, 1.92328037]]]]
Y = 
[[[[ 6.83778017,11.80664148,11.80664148,11.80664148, 7.32146590, 6.76203895, 6.76203895],
   [ 6.85141257,11.80664148,11.80664148,11.80664148, 7.32146590, 6.76203895, 6.762

In [26]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
pads = [0,0,0,1]
session = create_maxpool_model(x.shape, [1, 1, 6, 7], onnx_type, pads)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], pads)

X = 
[[[[ 3.77215090, 4.75723838, 8.55781965, 0.78615983, 0.12722420,-1.06758180, 1.72740480, 2.67392968],
   [ 5.82006760, 7.82941061, 7.07331734, 3.18499969, 2.08565643, 1.86802229, 3.76434712, 5.02928733],
   [ 5.10946681, 4.26304666, 7.60508046,-0.52712479, 0.94497943,-2.72429789, 4.82980308, 1.43855883],
   [ 0.30282386, 6.97465706, 2.66909992,-1.19734410, 1.78544601, 4.08093680, 0.90952255, 5.03971128],
   [ 1.87309120, 2.21011854, 0.53187335, 1.92701634, 5.09813924, 3.41752264, 1.56399777,-0.78419870],
   [ 3.27418685, 0.39654103,-0.41107836, 1.76179774, 8.16411925, 2.75642098,-1.24161395, 3.86984014],
   [-1.84656724, 1.99188346, 3.25518986,-2.91215305, 5.54321520, 4.55729149, 0.06518238, 8.22609852],
   [-0.71691664,-0.59373537, 3.78832115, 3.45530158, 4.62909760, 6.32596129, 5.05374653, 3.49156719]]]]
Y = 
[[[[8.55781965,8.55781965,8.55781965,3.18499969,4.82980308,5.02928733,5.02928733],
   [7.82941061,7.82941061,7.60508046,4.08093680,4.82980308,5.03971128,5.03971128],
   [7.

In [34]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.INT8
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
x_int8 = np.clip(np.round(x), -128, 127).astype(np.int8)
pads = [0,0,0,0]
session = create_maxpool_model(x_int8.shape, [1, 1, 6, 6], onnx_type, pads)
do_maxpool(x_int8, session)
MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[ 2, 3,-4, 7,-1, 4, 4, 5],
   [ 3, 6, 0, 2, 2, 2, 8, 3],
   [ 1, 7, 1, 6,-1, 1, 6, 2],
   [ 2, 0, 0, 4, 3, 1, 3, 5],
   [ 5, 6, 2, 5,-1, 3, 1, 3],
   [ 0, 6, 2,-4, 0, 1, 5, 4],
   [ 1, 0, 0, 3, 8, 4, 4, 3],
   [ 2, 9, 6,-1, 4, 1, 6, 3]]]]
Y = 
[[[[7,7,7,7,8,8],
   [7,7,6,6,8,8],
   [7,7,6,6,6,6],
   [6,6,5,5,5,5],
   [6,6,8,8,8,5],
   [9,9,8,8,8,6]]]]
Indices = 
[[[[17, 3, 3, 3,14,14],
   [17,17,19,19,14,14],
   [17,17,19,19,22,22],
   [33,33,35,35,46,31],
   [33,33,52,52,52,46],
   [57,57,52,52,52,62]]]]
MaxPool: X_p = 
[[[[ 2.00000000, 3.00000000,-4.00000000, 7.00000000,-1.00000000, 4.00000000, 4.00000000, 5.00000000],
   [ 3.00000000, 6.00000000, 0.00000000, 2.00000000, 2.00000000, 2.00000000, 8.00000000, 3.00000000],
   [ 1.00000000, 7.00000000, 1.00000000, 6.00000000,-1.00000000, 1.00000000, 6.00000000, 2.00000000],
   [ 2.00000000, 0.00000000, 0.00000000, 4.00000000, 3.00000000, 1.00000000, 3.00000000, 5.00000000],
   [ 5.00000000, 6.00000000, 2.00000000, 5.00000000,-1.00

In [46]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.INT8
x = np.random.normal(10, 4, size=(1, 1, 8, 8))
x_int8 = np.clip(np.round(x), -128, 127).astype(np.int8)
pads = [1,1,1,1]
session = create_maxpool_model(x_int8.shape, [1, 1, 8, 8], onnx_type, pads)
do_maxpool(x_int8, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[12,14,14,-1,15, 5,12,20],
   [ 7, 3,10,10,13,-3, 9, 8],
   [ 4,11,10, 3, 9,16,15,17],
   [ 4,13,18, 9,15,13, 9,13],
   [16,15, 9, 5, 7, 4,18,11],
   [ 5, 4, 6,11,13,16, 6,10],
   [17,12,18,11, 3,12,10,11],
   [ 9, 4,17, 5,10,12,13, 6]]]]
Y = 
[[[[14,14,14,15,15,15,20,20],
   [14,14,14,15,16,16,20,20],
   [13,18,18,18,16,16,17,17],
   [16,18,18,18,16,18,18,18],
   [16,18,18,18,16,18,18,18],
   [17,18,18,18,16,18,18,18],
   [17,18,18,18,16,16,16,13],
   [17,18,18,18,12,13,13,13]]]]
Indices = 
[[[[ 1, 1, 1, 4, 4, 4, 7, 7],
   [ 1, 1, 1, 4,21,21, 7, 7],
   [25,26,26,26,21,21,23,23],
   [32,26,26,26,21,38,38,38],
   [32,26,26,26,45,38,38,38],
   [48,50,50,50,45,38,38,38],
   [48,50,50,50,45,45,45,62],
   [48,50,50,50,53,62,62,62]]]]


In [47]:
onnx_type = onnx.TensorProto.INT8
data = [[[[ 1, 2, 5, 0, 0, 0, 0, 0],
          [ 3, 4, 6, 0, 0, 0, 0, 0],
          [ 7, 8, 9, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0]]]]
x = np.array(data, dtype=np.int8)
pads = [1,1,1,1]
session = create_maxpool_model(x_int8.shape, [1, 1, 8, 8], onnx_type, pads)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[1,2,5,0,0,0,0,0],
   [3,4,6,0,0,0,0,0],
   [7,8,9,0,0,0,0,0],
   [0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0]]]]
Y = 
[[[[4,6,6,6,0,0,0,0],
   [8,9,9,9,0,0,0,0],
   [8,9,9,9,0,0,0,0],
   [8,9,9,9,0,0,0,0],
   [0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0]]]]
Indices = 
[[[[ 9,10,10,10, 3, 4, 5, 6],
   [17,18,18,18, 3, 4, 5, 6],
   [17,18,18,18,11,12,13,14],
   [17,18,18,18,19,20,21,22],
   [24,24,25,26,27,28,29,30],
   [32,32,33,34,35,36,37,38],
   [40,40,41,42,43,44,45,46],
   [48,48,49,50,51,52,53,54]]]]


In [48]:
onnx_type = onnx.TensorProto.INT8
data = [[[[ -1, -2, 5, 0, 0, 0, 0, 0],
          [ -3, -4, 6, 0, 0, 0, 0, 0],
          [ 7, 8, 9, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0]]]]
x = np.array(data, dtype=np.int8)
pads = [1,1,1,1]
session = create_maxpool_model(x_int8.shape, [1, 1, 8, 8], onnx_type, pads)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[-1,-2, 5, 0, 0, 0, 0, 0],
   [-3,-4, 6, 0, 0, 0, 0, 0],
   [ 7, 8, 9, 0, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0]]]]
Y = 
[[[[-1, 6, 6, 6, 0, 0, 0, 0],
   [ 8, 9, 9, 9, 0, 0, 0, 0],
   [ 8, 9, 9, 9, 0, 0, 0, 0],
   [ 8, 9, 9, 9, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0],
   [ 0, 0, 0, 0, 0, 0, 0, 0]]]]
Indices = 
[[[[ 0,10,10,10, 3, 4, 5, 6],
   [17,18,18,18, 3, 4, 5, 6],
   [17,18,18,18,11,12,13,14],
   [17,18,18,18,19,20,21,22],
   [24,24,25,26,27,28,29,30],
   [32,32,33,34,35,36,37,38],
   [40,40,41,42,43,44,45,46],
   [48,48,49,50,51,52,53,54]]]]


In [50]:
onnx_type = onnx.TensorProto.INT8
data = [[[[ -128, -128, 5, 0, 0, 0, 0, 0],
          [ -128, -128, 6, 0, 0, 0, 0, 0],
          [ 7, 8, 9, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0],
          [ 0, 0, 0, 0, 0, 0, 0, 0]]]]
x = np.array(data, dtype=np.int8)
pads = [1,1,1,1]
session = create_maxpool_model(x_int8.shape, [1, 1, 8, 8], onnx_type, pads)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[-128,-128,   5,   0,   0,   0,   0,   0],
   [-128,-128,   6,   0,   0,   0,   0,   0],
   [   7,   8,   9,   0,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0]]]]
Y = 
[[[[-128,   6,   6,   6,   0,   0,   0,   0],
   [   8,   9,   9,   9,   0,   0,   0,   0],
   [   8,   9,   9,   9,   0,   0,   0,   0],
   [   8,   9,   9,   9,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0],
   [   0,   0,   0,   0,   0,   0,   0,   0]]]]
Indices = 
[[[[-9,10,10,10, 3, 4, 5, 6],
   [17,18,18,18, 3, 4, 5, 6],
   [17,18,18,18,11,12,13,14],
   [17,18,18,18,19,20,21,22],
   [24,24,25,26,27,28,29,30],
   [32,32,33,34,35,36,37,38],
   [40,40,41,42,43,44,45,46],
   [48,48,49,50,51,52,53,54]]

## No nominal cases (nan and inf values)

In [4]:
# Case NN1:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ np.nan, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, 1.67840883]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[        nan, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924, 1.67840886]]]]
Y = 
[[[[5.17326689,5.17326689,4.88280201,4.88280201,6.27754688,6.27754688,8.06596088,8.06596088],
   [6.38371611,6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.065960

2026-02-02 15:27:49.763249336 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.
2026-02-02 15:27:49.763272334 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.


In [5]:
# Case NN1:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, np.nan]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,        nan]]]]
Y = 
[[[[5.17326689,5.17326689,4.88280201,4.88280201,6.27754688,6.27754688,8.06596088,8.06596088],
   [6.38371611,6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.065960

2026-02-02 15:27:49.778044123 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.
2026-02-02 15:27:49.778061393 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.


In [6]:
# Case NN1:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ np.nan, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, np.nan,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, np.nan]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[        nan, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402,        nan,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,        nan]]]]
Y = 
[[[[5.17326689,5.17326689,4.88280201,4.88280201,6.27754688,6.27754688,8.06596088,8.06596088],
   [6.38371611,6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.065960

2026-02-02 15:27:49.793867967 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.
2026-02-02 15:27:49.793883488 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.


In [7]:
# Case NN2:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, np.inf]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,        inf]]]]
Y = 
[[[[5.17326689,5.17326689,4.88280201,4.88280201,6.27754688,6.27754688,8.06596088,8.06596088],
   [6.38371611,6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.065960

2026-02-02 15:27:49.809743684 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.
2026-02-02 15:27:49.809760852 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.


In [8]:
# Case NN3:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, -np.inf]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,       -inf]]]]
Y = 
[[[[5.17326689,5.17326689,4.88280201,4.88280201,6.27754688,6.27754688,8.06596088,8.06596088],
   [6.38371611,6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.065960

2026-02-02 15:27:49.824560261 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.
2026-02-02 15:27:49.824577736 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.


In [9]:
# Case NN4:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722,  np.nan, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, -np.inf]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726,        nan, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,       -inf]]]]
Y = 
[[[[5.17326689,5.17326689,4.88280201,4.88280201,6.27754688,6.27754688,8.06596088,8.06596088],
   [6.38371611,6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.065960

2026-02-02 15:27:49.841950445 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.
2026-02-02 15:27:49.841967019 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.


In [10]:
# Case NN4:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722,  np.nan, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, np.inf]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726,        nan, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,        inf]]]]
Y = 
[[[[5.17326689,5.17326689,4.88280201,4.88280201,6.27754688,6.27754688,8.06596088,8.06596088],
   [6.38371611,6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.065960

2026-02-02 15:27:49.858876465 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.
2026-02-02 15:27:49.858892965 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,8,8} target:{1,1,6,6}. Falling back to lenient merge.
